# DetectNet V2 Model Conversion

```
SPDX-FileCopyrightText: Copyright 2024 Arm Limited and/or its affiliates <open-source-office@arm.com>
SPDX-License-Identifier: MIT
```

This notebook can be used to convert a pre-trained DetectNet V2 model that is in ONNX format to TensorFlow Lite format to run on an Arm Cortex-A CPU or Arm Ethos-U NPU on a Linux based IoT platform.

## Upload ONNX model

Place the ONNX model in the notebook's directory or upload it the the Google Colab instance. Then update the variable value below with the model's filename.

In [ ]:
import os

INPUT_MODEL = "model_name.onnx"

INPUT_MODEL_ROOT, INPUT_MODEL_EXT = os.path.splitext(INPUT_MODEL)

## Rename inputs and outputs using ONNX.

This section will remove the `:0` suffixes from the model's inputs and outputs.

Install ONNX

In [ ]:
!pip install onnx

Use the ONNX library to create a modfied model with the renamed inputs and outputs.

In [ ]:
import onnx

onnx_model = onnx.load(INPUT_MODEL)

# input and output names to remove :0 suffix from
suffix = ':0'

graph_input_names = [input.name for input in onnx_model.graph.input]
graph_output_names = [output.name for output in onnx_model.graph.output]

print('graph_input_names =', graph_input_names)
print('graph_output_names =', graph_output_names)

for input in onnx_model.graph.input:
	input.name = input.name.removesuffix(suffix)

for output in onnx_model.graph.output:
	output.name = output.name.removesuffix(suffix)

for node in onnx_model.graph.node:
	for i in range(len(node.input)):
		if node.input[i] in graph_input_names:
			node.input[i] = node.input[i].removesuffix(suffix)

	for i in range(len(node.output)):
		if node.output[i] in graph_output_names:
			node.output[i] = node.output[i].removesuffix(suffix)

MODIFIED_ONNX_MODEL = f'{INPUT_MODEL_ROOT}_mod{INPUT_MODEL_EXT}'
onnx.save(onnx_model, MODIFIED_ONNX_MODEL)

## Convert model to OpenVINO format

Install OpenVINO

In [ ]:
!pip install openvino_dev

Use `mo` command to convert ONNX model to OpenVINO format

In [ ]:
!mo \
  --input_model {MODIFIED_ONNX_MODEL} \
  --input_shape [1,3,544,960] \
  --output_dir {INPUT_MODEL_ROOT}_openvino \
  --compress_to_fp16=False

## Convert OpenVINO model to TensorFlow

Install `openvino2tensorflow`

In [ ]:
!pip install openvino2tensorflow

Use `openvino2tensorflow` command to convert OpenVINO model. This will change the model from NCHW to NHWC format.

In [ ]:
!openvino2tensorflow \
  --model_path {INPUT_MODEL_ROOT}_openvino/{INPUT_MODEL_ROOT}_mod.xml \
  --model_output_path {INPUT_MODEL_ROOT}_tensorflow \
  --non_verbose \
  --output_saved_model

## Convert TensorFlow model to TensorFlow Lite format


Install TensorFlow

In [ ]:
!pip install tensorflow

Use TensorFlow to quantize the model with random representative data and convert to TensorFlow Lite format.


In [ ]:
import tensorflow as tf
import numpy as np

converter = tf.lite.TFLiteConverter.from_saved_model(f'{INPUT_MODEL_ROOT}_tensorflow')

tflite_model = converter.convert()

def representative_dataset():
  for _ in range(10):
    yield [
        tf.random.uniform((1, 544, 960, 3))
    ]

converter.optimizations = [
    tf.lite.Optimize.DEFAULT
]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.float32
converter.representative_dataset = representative_dataset

tflite_quant_model = converter.convert()

with open(f'{INPUT_MODEL_ROOT}.tflite', 'wb') as f:
    f.write(tflite_quant_model)


## Compile the TensorFlow Lite model with Vela

Install the `vela` compiler

In [ ]:
!pip install ethos-u-vela

Compile the quantized TensorFlow Lite model

In [ ]:
!vela \
  --config Arm/vela.ini \
  --accelerator-config ethos-u65-256 \
  --system-config Ethos_U65_High_End \
  --memory-mode Dedicated_Sram \
  --output-dir . \
  {INPUT_MODEL_ROOT}.tflite